# New Code

In [2]:
import datetime as dt 
import requests as rq
import json
import pandas as pd 
from pandasql import sqldf
import time
import numpy as np
from memory_profiler import profile

In [6]:
#url1="https://odre.opendatasoft.com/api/records/1.0/search/?dataset=consommation-quotidienne-brute-regionale&q=&sort=-consommation_brute_gaz_grtgaz&facet=date_heure&facet=code_insee_region&facet=region"
#url2="https://opendata.agenceore.fr/api/records/1.0/search/?dataset=production-demi-horaire-agregee-par-region&q=&sort=-horodate&facet=horodate&facet=region&facet=grd" 
#url22='https://tinyurl.com/4nhvpv2m'

list_api_1=[ 'region', 'code_insee_region','date_heure', 'date', 'heure', 
             'consommation_brute_electricite_rte', 'consommation_brute_totale', 'consommation_brute_gaz_totale']

list_api_2=['region','code','horodate',
            'energie_injectee', 'nb_points_injection' ]

# Adapters

In [7]:
class EnergyAdapter1:
    def __init__(self,list_of_tables):
        """ The adapater for the first API Energy"""
        self.base_url = 'https://odre.opendatasoft.com/api/records/1.0/search/?dataset=consommation-quotidienne-brute-regionale&q=&sort=-consommation_brute_gaz_grtgaz&facet=date_heure&facet=code_insee_region&facet=region'
        self.list_of_tables = list_of_tables
    def get_energy_production(self):
        lista=self.list_of_tables
        response = rq.get(self.base_url)
        Repense = response.content
        parse_json = json.loads(Repense)
        n=parse_json['records'].__len__()
        list_data=[x for x in parse_json['records']]
        list_dic,df={},{}
        for i in range(n):
            list_dic[i] = {key: list_data[i]["fields"][key] for key in lista}
        data=pd.DataFrame.from_dict(list_dic[0], orient='index').T
        for i in range(1,n):
            df[i]=pd.DataFrame.from_dict(list_dic[i], orient='index').T
            data=pd.concat([data,df[i]], axis=0)
        return data
    
class EnergyAdapter2:
    def __init__(self,list_of_tables):
        """ The adapater for the second API of Energy """
        self.base_url = "https://opendata.agenceore.fr/api/records/1.0/search/?dataset=production-demi-horaire-agregee-par-region&q=&sort=-horodate&facet=horodate&facet=region&facet=grd"
        self.list_of_tables = list_of_tables
    def get_energy_production(self):
        return EnergyAdapter1.get_energy_production(self)

# Mediator

In [8]:
class InformationMediator:
    def __init__(self, energy_adapter1, energy_adapter2):
        self.energy_adapter1 = energy_adapter1
        self.energy_adapter2 = energy_adapter2
    
    def get_combined_info(self,common):
        energy_data1 = self.energy_adapter1.get_energy_production()
        energy_data2 = self.energy_adapter2.get_energy_production()
        return pd.merge(energy_data1, energy_data2, on = common)
    
    
    def get_API1_info(self):
        return pd.DataFrame(self.energy_adapter1.get_energy_production())
    
    def get_API2_info(self):
        return pd.DataFrame(self.energy_adapter2.get_energy_production())
    
    def get_API1_API2_info(self):
        API1_df=get_API1_info(self)
        API2_df=get_API2_info(self)
        return pd.merge(API1_df, API2_df, on = common)

# -------------------------------------------------------------------------

$\bowtie$ -> $\Pi$     -> $\sigma$  $\hspace{1cm}$ $\bowtie$ -> $\sigma$  -> $\Pi$

$\Pi$     -> $\bowtie$ -> $\sigma$  $\hspace{1cm}$  $\Pi$     -> $\sigma$  -> $\bowtie$

    La projection Π 
    La sélection 𝜎
    La jointure ⋈ 

    parfois on aura besoin de qlq cols et/ou api et pas la totalité donc il faut traiter aussi ça.
    il faut trouver la bonne parmi ces méthodes 

1) Jointure first 
## 1-1) $\bowtie$ ->  $\Pi$ -> $\sigma$

In [9]:
def method_11(Query):
    # Π : La projection : all cols
    energyAdapter1 = EnergyAdapter1(list_api_1)
    energyAdapter2 = EnergyAdapter2(list_api_2)
    # ⋈ : La jointure : all cols
    informationMediator = InformationMediator(energyAdapter1,energyAdapter2)
    df = informationMediator.get_combined_info('region')
    sqldf(Query)

Query=""" 
SELECT region, consommation_brute_electricite_rte, energie_injectee
FROM df
WHERE df.region='Pays de la Loire'"""

In [10]:
start_time = time.time()
method_11(Query)
elapsed_time11 = time.time() - start_time
print("Temps d'exécution : ", elapsed_time11, " secondes.")

Temps d'exécution :  3.9245944023132324  secondes.


## 1-2)  $\bowtie$ -> $\sigma$  -> $\Pi$

In [11]:
def method_12(Query):
    # Π : La projection : all cols
    energyAdapter1 = EnergyAdapter1(list_api_1)
    energyAdapter2 = EnergyAdapter2(list_api_2)
    # ⋈ : La jointure : all cols
    informationMediator = InformationMediator(energyAdapter1,energyAdapter2)
    dfff = informationMediator.get_combined_info('region')
    informationMediator.get_combined_info('region').head(3)
    # 𝜎 : La Selection 
    df_selection_step=dfff[dfff['region']=='Pays de la Loire']
    sqldf(Query)

Query=""" 
SELECT region, consommation_brute_electricite_rte, energie_injectee
FROM df_selection_step """

In [12]:
start_time = time.time()
method_12(Query)
elapsed_time12 = time.time() - start_time
print("Temps d'exécution : ", elapsed_time12, " secondes.")

Temps d'exécution :  3.80364990234375  secondes.


2) Projection first (<<COLUMNS)
# 2-1) $\Pi$     -> $\bowtie$ -> $\sigma$ 

In [13]:
def method_21(Query):
    start, end = 'SELECT','FROM'
    tables_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split(', ')
    list_api1, list_api2, common= [], [], []
    for word in tables_in_query:
        if (word in list_api_1) and (word in list_api_2):
            common.append(word)
            list_api1.append(word)
            list_api2.append(word)
        elif  word in list_api_1:
            list_api1.append(word)
        else:
            list_api2.append(word)
    # Π : La projection : qlq cols
    energyAdapter1 = EnergyAdapter1(list_api1)
    energyAdapter2 = EnergyAdapter2(list_api2)
    # ⋈ : La jointure : qlq cols
    informationMediator = InformationMediator(energyAdapter1,energyAdapter2)
    df=informationMediator.get_combined_info(common)
    # 𝜎 : La sélection 
    x=['Pays de la Loire']
    df.query(f" region == {x}")

Query=""" 
SELECT region, consommation_brute_electricite_rte, energie_injectee
FROM df1
INNER JOIN df2 ON df1.region = df2.region
WHERE df2.region='Pays de la Loire'"""

In [14]:
start_time = time.time()
method_21(Query)
elapsed_time21 = time.time() - start_time
print("Temps d'exécution : ", elapsed_time21, " secondes.")
#Temps d'exécution :  2.115198850631714  secondes.

Temps d'exécution :  2.2614502906799316  secondes.


# 2-2)  $\Pi$     -> $\sigma$  -> $\bowtie$

In [15]:
def method_22(Query):
    # Π : La projection : qlq cols
    start, end = 'SELECT', 'FROM'
    tables_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split(', ')
    list_api1, list_api2, common= [], [], []
    for word in tables_in_query:
        if (word in list_api_1) and (word in list_api_2):
            common.append(word)
            list_api1.append(word)
            list_api2.append(word)
        elif  word in list_api_1:
            list_api1.append(word)
        else:
            list_api2.append(word)
    energyAdapter1, energyAdapter2 = EnergyAdapter1(list_api1), EnergyAdapter2(list_api2)
    informationMediator = InformationMediator(energyAdapter1,energyAdapter2)
    df1, df2=informationMediator.get_API1_info() ,informationMediator.get_API2_info()  
    
    # 𝜎 : La sélection        
    start, end = "WHERE", ""
    tables_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split('=')[1]
    dff=df1[df1['region']==tables_in_query]
    dfff=df2[df2['region']==tables_in_query]

    # ⋈ : La jointure : qlq cols
    df=pd.merge(dff, dfff, on = "region")
    df

Query=""" 
SELECT region, consommation_brute_electricite_rte, energie_injectee
FROM df1
INNER JOIN df2 ON df1.region = df2.region
WHERE df2.region='Pays de la Loire'"""

In [16]:
start_time = time.time()
method_22(Query)
elapsed_time22 = time.time() - start_time
print("Temps d'exécution : ", elapsed_time22, " secondes.")

Temps d'exécution :  1.9012627601623535  secondes.


In [17]:
(elapsed_time11, elapsed_time12, elapsed_time21, elapsed_time22)

(3.9245944023132324, 3.80364990234375, 2.2614502906799316, 1.9012627601623535)

Temps d'exécution :
	⋈-> Π-> 𝜎           ⋈-> 𝜎-> Π           Π-> ⋈-> 𝜎           Π-> 𝜎-> ⋈
(3.8648767471313477,  4.503273248672485,   2.081739664077759,   1.9406847953796387)
(2.583775520324707,   3.8232362270355225,  1.386244535446167,   1.8930981159210205)
(3.42537522315979,    4.277031660079956,   2.1051204204559326,  2.066295862197876)
(3.3437840938568115,  3.0091187953948975,  1.464475393295288,   2.062783718109131)
(3.1498332023620605,  4.417710304260254,   1.871737003326416,   2.104344367980957)

(3.616210699081421,   3.5588901042938232,  1.476405143737793,   1.3976826667785645)
(2.8788163661956787,  4.071977138519287,   3.204068183898926,   1.9847311973571777)
(3.369443655014038,   3.1831319332122803,  1.8376247882843018,  1.6176726818084717)
(2.6496338844299316,  4.1961143016815186,  1.1578257083892822,  1.2491710186004639)
(2.0068414211273193,  2.613250970840454,   0.8801448345184326,  0.8789918422698975)

# 3-1)  $\sigma$   -> $\Pi$    -> $\bowtie$
	to do 

# 3-2)  $\sigma$   ->  $\bowtie$    -> $\Pi$
	to do

Exemple de queries:

'' Savoir la consommation d'énergie brute totale et la consommation brute d'électricité ''

1) query = "SELECT country, region, consommation_brute_totale, consommation_brute_electricite_rte FROM df"

'' Savoir la température et l'humidité sur Occitanie (Toulouse)''

2) query = "SELECT temp_celsius ,temperature, humidity, wind_speed FROM df"

'' Savoir la température et la consommation d'énergie brute totale sur Occitanie (Toulouse)''

3) query = "SELECT temp_celsius ,consommation_brute_totale FROM df"